In [17]:
import os
import re
import json
from pprint import pprint
from tqdm import tqdm

In [18]:
hangeul = ['가', '나', '다', '라', '마', '바', '사']
section_list = ['\u25b6', 'Q.', 'Guide>', '*']
for i in range(1, 10):
	section_list.append(str(i) + '.')
	section_list.append(str(i) + ')')
	section_list.append(str(i) + ' .')
	section_list.append(str(i) + ' )')
	section_list.append('Essay ' + str(i))
	section_list.append('Essay' + str(i))
for h in hangeul:
	section_list.append(h + '.')
	section_list.append(h + ')')
	section_list.append(h + ' .')
	section_list.append(h + ' )')

In [19]:
data_dir = '../data_re_labelled/'
save_dir = '../data_question_seperated/'
data_list = os.listdir(data_dir)
already_exist = os.listdir(save_dir)
is_already_exist = list()
cnt = 1

In [20]:
for d in tqdm(already_exist):
	file_name = save_dir + d
	with open(file_name) as file:
		json_file = json.load(file)
		# --- #
		id = json_file['id']
		is_already_exist.append(id)

100%|██████████| 11795/11795 [00:01<00:00, 6244.37it/s]


In [21]:
def num_plus_comma(cnt: int) -> str:
	cnt = str(cnt)
	cnt = ',' + cnt[1:]
	return cnt

In [22]:
def extract_question(li: list) -> list:
	ret = list()
	for l in li:
		flag = False
		for section in section_list:
			if l[:5].find(section) != -1:
				ret.append(l)
				break
			for i in range(100, 2500):
				chk = str(i) + '자'
				if l.find(chk) != -1:
					ret.append(l)
					flag = True
					break
				chk = str(i) + ' 자'
				if l.find(chk) != -1:
					ret.append(l)
					flag = True
					break
				chk = str(i) + 'byte'
				if l.find(chk) != -1:
					ret.append(l)
					flag = True
					break
				chk = str(i) + ' byte'
				if l.find(chk) != -1:
					ret.append(l)
					flag = True
					break
				if i >= 1000:
					count = num_plus_comma(i)
					chk = count + '자'
					if l.find(chk) != -1:
						ret.append(l)
						flag = True
						break
					chk = count + ' 자'
					if l.find(chk) != -1:
						ret.append(l)
						flag = True
						break
					chk = count + 'byte'
					if l.find(chk) != -1:
						ret.append(l)
						flag = True
						break
					chk = count + ' byte'
					if l.find(chk) != -1:
						ret.append(l)
						flag = True
						break
			if flag is True:
				break
	return ret

In [23]:
def list_sentence(content: str) -> list:
    content = content.replace('\r', '\n')
    li = content.split('\n')
    for i in range(len(li) - 1):
        if len(li[i + 1]) == 0:
            li[i] += '\n'
    for l in li[:]:
        if len(l) == 0:
            li.remove(l)
    return li


In [24]:
def save_file(d, idx):
	file_name = data_dir + d
	with open(file_name) as file:
		json_file = json.load(file)
		# --- #
		id = json_file['id']
		if id in is_already_exist:
			return

		q, l = question_sep(json_file['content'])
		if len(q) > 0:
			match_question_answer(q=q, l=l, json_file=json_file)

		# --- #


In [25]:
def match_question_answer(q: list, l: list, json_file: dict):
	sub_cnt = 0
	q_idx = list()
	for idx, data in enumerate(l):
		if data in q:
			q_idx.append(idx)
	q_idx.append(len(l))

	for idx in range(0, len(q_idx) - 1):
		sub_cnt += 1
		new_json = dict()
		question = l[q_idx[idx]]
		answer = ' '.join(l[q_idx[idx] + 1:q_idx[idx + 1]])
		new_json['id'] = json_file['id']
		new_json['sub_id'] = sub_cnt
		new_json['question'] = question
		new_json['answer'] = answer
		new_json['license'] = json_file['license']
		new_json['activity'] = json_file['activity']
		new_json['career'] = json_file['career']
		new_json['languageScore'] = json_file['languageScore']
		new_json['grades'] = json_file['grades']
		new_json['major'] = json_file['major']
		new_json['organizaitonName'] = json_file['organizationName']
		new_json['role'] = json_file['role']
		new_json['university'] = json_file['university']

		# pprint(new_json)
		# print('---------------------------------------------------------------------------------------------------------')
		
		save_file_name = save_dir + 'data_qeustion_seperated' + str(json_file['id']) + '-' + str(sub_cnt) + '.json'
		with open(save_file_name, 'w') as f:
			json.dump(new_json, f, ensure_ascii=False, indent=4, sort_keys=True)


In [26]:
def question_sep(content: str) -> str:
    li = list_sentence(content=content)
    question = extract_question(li)
    return question, li

In [27]:
for d in tqdm(data_list):
    save_file(d, cnt)
    cnt += 1

100%|██████████| 9588/9588 [14:55:14<00:00,  5.60s/it]  
